In [1]:
import pandas as pd
import numpy as np

In [2]:
city_stats = pd.read_parquet(
    "../20_intermediate_data/city_statistics.parquet"
).reset_index(drop=True)


In [3]:
systems = pd.read_parquet(
    "../20_intermediate_data/system_statistics.parquet"
).reset_index(drop=True)


In [4]:
trips = pd.read_parquet("../20_intermediate_data/trips.parquet").reset_index(drop=True)


In [5]:
weather = pd.read_parquet("../20_intermediate_data/weather.parquet").reset_index(
    drop=True
)
weather = weather.fillna(0)


In [6]:
cities = city_stats.merge(systems, how="outer", on=["City", "Year"], validate = "m:1")
cities["Average Fare"] = cities["Average Fare"] / cities["CPI"]
cities["Gas Price"] = cities["Gas Price"] / cities["CPI"]


In [7]:
trips.drop("Agency", axis=1, inplace=True)
cities.drop("Agency Name", axis=1, inplace=True)


In [8]:
cities = cities.merge(
    trips, how="outer", on=["City", "Month", "Year", "Month Numeric"], validate="1:1"
)


In [9]:
weather.loc[weather.loc[:, "City"] == "Washington, D.C", "City"] = "Washington, D.C."


In [10]:
final = cities.merge(
    weather, how="outer", on=["City", "Year", "Month Numeric"], validate="1:1"
)


In [11]:
final["Uber"] = 0


In [12]:
final["Date"] = pd.to_datetime(
    final["Year"].astype(str) + "-" + final["Month Numeric"].astype(str) + "-01"
)


In [13]:
final.loc[
    (final.loc[:, "Date"] > "2011-09-01") & (final.loc[:, "City"] == "Chicago"),
    "Uber",
] = 1
final.loc[
    (final.loc[:, "Date"] > "2012-08-01") & (final.loc[:, "City"] == "Atlanta"), "Uber"
] = 1
final.loc[
    (final.loc[:, "Date"] > "2011-10-01") & (final.loc[:, "City"] == "Boston"), "Uber"
] = 1
final.loc[
    (final.loc[:, "Date"] > "2011-06-01") & (final.loc[:, "City"] == "New York City"),
    "Uber",
] = 1
final.loc[
    (final.loc[:, "Date"] > "2012-01-01")
    & (final.loc[:, "City"] == "Washington, D.C."),
    "Uber",
] = 1
final.loc[
    (final.loc[:, "Date"] > "2015-01-01") & (final.loc[:, "City"] == "Portland"), "Uber"
] = 1
final.loc[
    (final.loc[:, "Date"] > "2012-04-01") & (final.loc[:, "City"] == "Los Angeles"),
    "Uber",
] = 1
final.loc[
    (final.loc[:, "Date"] > "2010-06-01") & (final.loc[:, "City"] == "San Francisco"),
    "Uber",
] = 1
final.loc[
    (final.loc[:, "Date"] > "2012-07-01") & (final.loc[:, "City"] == "Philadelphia"),
    "Uber",
] = 1


In [14]:
final["Season"] = np.NaN
winter = [12, 1, 2]
spring = [3, 4, 5]
summer = [6, 7, 8]
fall = [9, 10, 11]


In [15]:
final.loc[final.loc[:, "Month Numeric"].isin(winter), "Season"] = "Winter"
final.loc[final.loc[:, "Month Numeric"].isin(spring), "Season"] = "Spring"
final.loc[final.loc[:, "Month Numeric"].isin(summer), "Season"] = "Summer"
final.loc[final.loc[:, "Month Numeric"].isin(fall), "Season"] = "Fall"


In [16]:
final.to_csv("../25_final_data/public_transportation.csv")